In [14]:
import pandas as pd

In [15]:
df = pd.read_csv('./raw_data/sample_media_spend_data.csv')
type(df)

pandas.core.frame.DataFrame

In [16]:

import pandas as pd
import numpy as np
import pickle
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
from data_processing import *

with open('config.yaml') as infile:
    config = yaml.safe_load(infile)
features_to_use = config["features_to_use"]
columns_to_scale = config["columns_to_scale"]

# Page config


# Load the model and data

def load_model():
    """Uses pickle to load trained model. 

    Returns:
        sklearn.linear_model._base.LinearRegression : Trained MMM model
    """
    return pickle.load(open('analysis/model.pkl', 'rb'))


def load_data():
    """Loading the sample_media_spend_data csv as df,
       performing basic preprocessing.

    Returns:
        pandas.core.frame.DataFrame: loaded dataset as df
    """
    df = pd.read_csv('raw_data/sample_media_spend_data.csv')
    df = df.reset_index(drop = True)
    df.columns = map(str.lower, df.columns)
    return df

model = load_model()
df = load_data()


selected_division = sorted(df['division'].unique())[0]
selected_division


'A'

In [17]:

# Filter data for selected division
df_filtered = df[df['division'] == selected_division].copy()
df_filtered.head()

,division,calendar_week,paid_views,organic_views,google_impressions,email_impressions,facebook_impressions,affiliate_impressions,overall_views,sales
0,A,1/6/2018,392,422,408,349895.0107,73580,12072,682,59417
1,A,1/13/2018,787,904,110,506270.2176,11804,9499,853,56806
2,A,1/20/2018,81,970,742,430042.1538,52232,17048,759,48715
3,A,1/27/2018,25,575,65,417745.6658,78640,10207,942,72047
4,A,2/3/2018,565,284,295,408505.8012,40561,5834,658,56235


In [18]:
# ROI Analysis Tab
# with tab1:

# Calculate channel ROI
channels = ['paid_views', 'google_impressions', 'email_impressions', 
            'facebook_impressions', 'affiliate_impressions']


In [36]:
channel = channels[2]
channel

'email_impressions'

In [37]:
roi_data = []


In [38]:

avg_spend = df_filtered[channel].mean()
avg_spend

np.float64(389433.0160858407)

In [39]:

# Create two scenarios for ROI calculation
base_scenario = df_filtered.copy()
increased_scenario = df_filtered.copy()
increased_scenario[channel] = increased_scenario[channel] * 1.1  # 10% increase
increased_scenario[channel].mean()

np.float64(428376.31769442477)

In [40]:
# all the features we want to use are part of colummns to scale
subset = []
for feature in features_to_use:
    subset.append(feature in columns_to_scale)
all(subset)

True

In [41]:
# NOTE: preprocessing: not sure exactly where to put this...

base_scenario = preprocess_df(base_scenario)
increased_scenario = preprocess_df(increased_scenario)
increased_scenario.head()

,division,calendar_week,paid_views,organic_views,google_impressions,email_impressions,facebook_impressions,affiliate_impressions,overall_views,sales,date,day,week,month,year
0,A,1/6/2018,392,422,408,384884.51177,73580,12072,682,59417,2018-01-06,6,1,1,2018
1,A,1/13/2018,787,904,110,556897.23936,11804,9499,853,56806,2018-01-13,13,2,1,2018
2,A,1/20/2018,81,970,742,473046.36918,52232,17048,759,48715,2018-01-20,20,3,1,2018
3,A,1/27/2018,25,575,65,459520.23238,78640,10207,942,72047,2018-01-27,27,4,1,2018
4,A,2/3/2018,565,284,295,449356.38132,40561,5834,658,56235,2018-02-03,3,5,2,2018


In [42]:

base_scenario = encoding_categorical_features(base_scenario, columns_to_scale)
increased_scenario = encoding_categorical_features(increased_scenario, columns_to_scale)

# Scale data
base_scenario[columns_to_scale] = scaler.fit_transform(base_scenario[columns_to_scale])
increased_scenario[columns_to_scale] = scaler.transform(increased_scenario[columns_to_scale])


In [43]:
base_scenario.head()

,paid_views,organic_views,google_impressions,email_impressions,facebook_impressions,affiliate_impressions,overall_views,sales,day,week,...,division_Q,division_R,division_S,division_T,division_U,division_V,division_W,division_X,division_Y,division_Z
0,0.005479,0.010373,0.000153,0.196934,0.064313,0.597574,0.006643,59417,0.166667,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.011296,0.022509,0.000020,0.390865,0.010157,0.389486,0.008668,56806,0.400000,0.019608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000898,0.024171,0.000303,0.296330,0.045598,1.000000,0.007555,48715,0.633333,0.039216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000074,0.014226,0.000000,0.281080,0.068749,0.446745,0.009722,72047,0.866667,0.058824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.008027,0.006899,0.000103,0.269621,0.035367,0.093085,0.006359,56235,0.066667,0.078431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
increased_scenario.head()

,paid_views,organic_views,google_impressions,email_impressions,facebook_impressions,affiliate_impressions,overall_views,sales,day,week,...,division_Q,division_R,division_S,division_T,division_U,division_V,division_W,division_X,division_Y,division_Z
0,0.005479,0.010373,0.000153,0.240327,0.064313,0.597574,0.006643,59417,0.166667,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.011296,0.022509,0.000020,0.453651,0.010157,0.389486,0.008668,56806,0.400000,0.019608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000898,0.024171,0.000303,0.349662,0.045598,1.000000,0.007555,48715,0.633333,0.039216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000074,0.014226,0.000000,0.332888,0.068749,0.446745,0.009722,72047,0.866667,0.058824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.008027,0.006899,0.000103,0.320283,0.035367,0.093085,0.006359,56235,0.066667,0.078431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
channel in features_to_use # if false, no ROI increase because correlation for this channel was very high with some other channel

False

In [ ]:

# features_scaled = scaler.transform(base_scenario[channels])
# features_scaled_increased = scaler.transform(increased_scenario[channels])

# Get predictions
base_pred = model.predict(base_scenario[features_to_use])
increased_pred = model.predict(increased_scenario[features_to_use])


In [46]:
base_pred

array([ 94422.44807242,  87279.25206604,  90324.47933173,  91905.526556  ,
        94535.16990778,  93180.37761143,  99576.31216234,  87540.91959931,
        83927.36920631,  89894.74183172,  81705.16197288,  85341.37581691,
        81468.51611055,  93601.91327665,  91062.74942889,  87813.31937287,
        86183.23749302,  92418.24178655,  92367.14904421,  87572.15985626,
        87636.04535482,  87888.56587508,  90279.69582508,  84305.27241564,
        82168.6433773 ,  82767.2165432 ,  81939.08148878,  94001.39454385,
        87586.48195472,  83140.22526525,  93836.2343965 ,  93830.77036901,
       100331.2521973 , 113523.85123585,  87781.41892358,  90739.3961723 ,
        82946.65127908,  85007.19283535,  84403.9103118 , 137187.04262524,
       137186.75480744, 138384.8941928 , 136781.62212252, 190067.37361302,
       199119.0712827 , 217201.14590049, 264581.70384367, 214350.28048451,
       194016.13224243, 199117.27456505, 189522.89658954, 187849.02828593,
        93529.36638213,  

In [47]:
increased_pred

array([ 94422.44807242,  87279.25206604,  90324.47933173,  91905.526556  ,
        94535.16990778,  93180.37761143,  99576.31216234,  87540.91959931,
        83927.36920631,  89894.74183172,  81705.16197288,  85341.37581691,
        81468.51611055,  93601.91327665,  91062.74942889,  87813.31937287,
        86183.23749302,  92418.24178655,  92367.14904421,  87572.15985626,
        87636.04535482,  87888.56587508,  90279.69582508,  84305.27241564,
        82168.6433773 ,  82767.2165432 ,  81939.08148878,  94001.39454385,
        87586.48195472,  83140.22526525,  93836.2343965 ,  93830.77036901,
       100331.2521973 , 113523.85123585,  87781.41892358,  90739.3961723 ,
        82946.65127908,  85007.19283535,  84403.9103118 , 137187.04262524,
       137186.75480744, 138384.8941928 , 136781.62212252, 190067.37361302,
       199119.0712827 , 217201.14590049, 264581.70384367, 214350.28048451,
       194016.13224243, 199117.27456505, 189522.89658954, 187849.02828593,
        93529.36638213,  

In [48]:
# Calculate ROI
sales_lift = (increased_pred - base_pred).mean()
sales_lift

np.float64(0.0)

In [49]:
# Simulating a 10% increase in investment 
investment_increase = avg_spend * 0.1
investment_increase

np.float64(38943.30160858407)

In [35]:

# ROI computed as change in sales over change in investment
roi = (sales_lift / investment_increase) if investment_increase > 0 else 0
roi

np.float64(-0.0877363018989934)